In [1]:
import os
import gc
import re
import sys
import math
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
train_temp = pd.read_csv('../tmp/train/train_temp.csv')
train_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286290 entries, 0 to 286289
Data columns (total 23 columns):
origin_name    286290 non-null object
brand_name     283473 non-null object
model_name     283473 non-null object
detail_name    283473 non-null object
cos_similar    283473 non-null float64
brand_slug     283473 non-null float64
model_slug     283473 non-null float64
detail_slug    283473 non-null float64
online_year    283473 non-null float64
energy         283473 non-null object
body           283473 non-null object
control        283473 non-null object
volume         283473 non-null float64
year           286187 non-null float64
month          286187 non-null float64
mile           286290 non-null float64
city           286290 non-null object
price_bn       283473 non-null float64
price          286290 non-null float64
create_time    286290 non-null object
domain         286290 non-null object
labels         271618 non-null object
url            286290 non-null object
dtyp

In [ ]:
################
# 1.长安欧尚 奔奔mini-e 无此车系
# 2.新凯 品牌 缺失车系
# 3.福田 拓陆者 品牌描述错误 为 福田乘用车(停售车)
# 4.海马 福仕达腾达 车系缺失

In [4]:
man = train_temp.loc[(train_temp['brand_name'].isnull()),:]
man.to_csv('../tmp/train/man.csv',index=False)

In [10]:
man = train_temp.loc[(train_temp['brand_slug'] == 45),:]
man

,origin_name,brand_name,model_name,detail_name,cos_similar,brand_slug,model_slug,detail_slug,online_year,energy,...,year,month,mile,city,price_bn,price,create_time,domain,labels,url
5458,Smart smart forfour 2016款 0.9T 66千瓦先锋版,smart,smart forfour,2016款 0.9T 66千瓦先锋版,1.000,45.0,1004.0,26155.0,2016.0,汽油,...,2017.0,1.0,1.5,贵阳,15.4800,10.50,2019-03-04 18:03:14,guazi.com,NaN,https://www.guazi.com/gy/3008144343x.htm
5459,Smart smart forfour 2016款 0.9T 66千瓦先锋版,smart,smart forfour,2016款 0.9T 66千瓦先锋版,1.000,45.0,1004.0,26155.0,2016.0,汽油,...,2017.0,6.0,0.6,武汉,15.4800,11.00,2019-03-04 18:03:15,guazi.com,"4s店保养,0过户",https://www.guazi.com/wh/3008196888x.htm
5460,Smart smart forfour 2016款 0.9T 66千瓦先锋版,smart,smart forfour,2016款 0.9T 66千瓦先锋版,1.000,45.0,1004.0,26155.0,2016.0,汽油,...,2017.0,9.0,1.3,重庆,15.4800,12.00,2019-03-04 18:03:24,guazi.com,0过户,https://www.guazi.com/cq/3008153122x.htm
5461,Smart smart forfour 2016款 0.9T 66千瓦先锋版,smart,smart forfour,2016款 0.9T 66千瓦先锋版,1.000,45.0,1004.0,26155.0,2016.0,汽油,...,2016.0,5.0,3.2,青岛,15.4800,10.00,2019-03-14 23:28:02,guazi.com,"4s店保养,质保",https://www.guazi.com/qd/3008327405x.htm
5462,Smart smart forfour 2016款 0.9T 66千瓦先锋版,smart,smart forfour,2016款 0.9T 66千瓦先锋版,1.000,45.0,1004.0,26155.0,2016.0,汽油,...,2016.0,9.0,2.5,绍兴,15.4800,10.00,2019-03-19 05:02:03,guazi.com,质保,https://www.guazi.com/shaoxing/3008428635x.htm
5463,Smart smart forfour 2016款 0.9T 66千瓦先锋版,smart,smart forfour,2016款 0.9T 66千瓦先锋版,1.000,45.0,1004.0,26155.0,2016.0,汽油,...,2017.0,8.0,1.2,宿迁,15.4800,11.00,2019-03-20 14:05:51,guazi.com,"4s店保养,质保",https://www.guazi.com/suqian/3008481192x.htm
5464,Smart smart forfour 2016款 0.9T 66千瓦先锋版,smart,smart forfour,2016款 0.9T 66千瓦先锋版,1.000,45.0,1004.0,26155.0,2016.0,汽油,...,2017.0,2.0,1.6,南京,15.4800,11.00,2019-03-21 18:09:30,guazi.com,质保,https://www.guazi.com/nj/3008510904x.htm
5465,Smart smart forfour 2016款 0.9T 66千瓦先锋版,smart,smart forfour,2016款 0.9T 66千瓦先锋版,1.000,45.0,1004.0,26155.0,2016.0,汽油,...,2017.0,9.0,0.9,大同,15.4800,10.00,2019-03-23 21:00:48,guazi.com,质保,https://www.guazi.com/datong/3008543181x.htm
5466,Smart smart forfour 2016款 0.9T 66千瓦极致版,smart,smart forfour,2016款 0.9T 66千瓦极致版,1.000,45.0,1004.0,24585.0,2016.0,汽油,...,2017.0,10.0,1.2,兰州,17.3800,10.20,2019-03-04 18:03:22,guazi.com,"4s店保养,0过户,急售,超值",https://www.guazi.com/lz/3006406828x.htm
5467,Smart smart forfour 2016款 0.9T 66千瓦极致版,smart,smart forfour,2016款 0.9T 66千瓦极致版,1.000,45.0,1004.0,24585.0,2016.0,汽油,...,2016.0,5.0,0.8,成都,17.3800,13.00,2019-03-14 23:28:01,guazi.com,质保,https://www.guazi.com/cd/3008342399x.htm


In [2]:
def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """
    text = df[column_name]
    text = text.lower()
    text = text.replace('ⅰ', 'i')
    text = text.replace('ⅱ', 'ii')
    text = text.replace('ⅲ', 'iii')
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace('【', '')
    text = text.replace('】', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('_', '')
    text = text.replace('-', '')
    text = text.replace('+', '')
    text = text.replace('—', '')
    text = text.replace('/', '')
    text = text.replace('“', '')
    text = text.replace('”', '')
    text = text.replace('!', '')
    text = text.replace('。', '')
    text = text.replace('＞', '')
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('》', '')
    text = text.replace('！', '')
    text = text.replace('／', '')
    text = text.replace('’', '')
    text = text.replace('－', '')
    text = text.replace('•', '')
    text = text.replace('×', '')
    text = text.replace('《', '')
    text = text.replace('＿', '')
    text = text.replace('®', '')
    text = text.replace('─', '')
    text = text.replace('、', '')
    text = text.replace(' ', '') 
    return text

### 一.生成词向量映射

In [3]:
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all['final_text'] = car_autohome_all['brand_name'] + car_autohome_all['model_name'] + car_autohome_all['detail_name']
car_autohome_all['final_text'] = car_autohome_all.apply(delete_str_useless,args=('final_text',),axis=1)

In [4]:
texts = []
train_final = []
generate_text_index = car_autohome_all.copy()
for i in range(0, len(generate_text_index)):
    temp = []
    texts.extend(generate_text_index['final_text'][i])
    x = generate_text_index['final_text'][i]
    temp.extend(x)
    train_final.append(temp)
    
# 创建Tokenizer
tokenizer = Tokenizer(num_words=len(set(texts)),char_level=True, lower=True)
tokenizer.fit_on_texts(train_final)
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

# 保存映射器和词典
with open('./tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

f = open('./word_index.txt', 'w')
f.write(str(word_index))
f.close()

Found 1057 unique tokens


### 二.生成标准库余弦向量

In [5]:
f = open('./word_index.txt', 'r', encoding='UTF-8')
temp = f.read()
word_index = eval(temp)

with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all['brand_name'] = car_autohome_all.apply(delete_str_useless,args=('brand_name',),axis=1)
car_autohome_all['model_name'] = car_autohome_all.apply(delete_str_useless,args=('model_name',),axis=1)

car_autohome_all['final_text'] = car_autohome_all['brand_name'] + car_autohome_all['model_name'] + car_autohome_all['detail_name']
car_autohome_all['final_text'] = car_autohome_all.apply(delete_str_useless,args=('final_text',),axis=1)
car_autohome_all['fill_vector'] = np.NaN

train_final = []
for i in range(0, len(car_autohome_all)):
    temp = []
    x = car_autohome_all['final_text'][i]
    temp.extend(x)
    train_final.append(temp)
train_sequences = tokenizer.texts_to_sequences(train_final)

fill_zero = [0 for i in range(0,len(word_index))]

for i in range(0,len(car_autohome_all)):  
    text = fill_zero.copy()
    for j in train_sequences[i]:
        text[j-1] = text[j-1] + 1
    car_autohome_all.loc[i,'fill_vector'] = str(text)
car_autohome_all.to_csv('./car_autohome_cos_vector.csv',index=False)

### 三.待匹配车源

In [27]:
def add_brand(brand):
    brand = brand.append(pd.DataFrame([['昌河','79'],['吉利','25'],['传祺','82'],['swm斯威','269'],['斯威','269'],['猎豹','78'],
                                      ['力帆','80'],['五菱','114'],['野马','111'],['mg','20'],['英致','192'],['汉腾','267'],
                                      ['吉奥','108'],['康迪','219'],['夏利','110'],['莲花','89'],['北汽','154']],columns=['brand_name','brand_slug']),sort=False).reset_index(drop=True)
    return brand

def find_brand(df):
    for i in range(0,len(brand)):
        if brand.loc[i,'brand_name'] == 'mg':
            match = re.findall('^mg',df['detail_name'])
            if len(match) > 0:
                return pd.Series([brand.loc[i,'brand_name'],brand.loc[i,'brand_slug']])
        else:           
            match = re.findall(brand.loc[i,'brand_name'],df['detail_name'])
            if len(match) > 0:
                return pd.Series([brand.loc[i,'brand_name'],brand.loc[i,'brand_slug']])
    return pd.Series([np.NaN,np.NaN])

def process_online_year(df):
    temp = re.findall('(\d\d\d\d)款',df['detail_name'])
    if len(temp) == 0:
        return np.NaN
    return temp[0]

def process_volume(df):
    temp = re.findall('(\d\.\d)',df['detail_name'])
    if len(temp) == 0:
        return np.NaN
    return temp[0]

def process_control(df):
    text = df['detail_name']
    text = text.lower()
    temp = re.findall('自动|手自|amt|无级|cvt|双离合|dct|dsg',text)
    if len(temp) > 0:
        return '自动'
    temp = re.findall('手动',text)
    if len(temp) > 0:
        return '手动'
    return np.NaN

def cal_brand_count(df):
    return len(df['brand_name'])

origin_train = pd.read_csv('./origin_train.csv')
origin_train = origin_train.loc[(origin_train['domain'] == 'guazi.com'),:].reset_index(drop=True).head(5000)
origin_train['online_year'] = origin_train.apply(process_online_year,axis=1)
origin_train = origin_train.loc[(origin_train['online_year'].notnull()),:].reset_index(drop=True)
origin_train['volume'] = origin_train.apply(process_volume,axis=1)
origin_train['control'] = origin_train.apply(process_control,axis=1)
origin_train['detail_name'] = origin_train.apply(delete_str_useless,args=('detail_name',),axis=1)

brand = car_autohome_all.loc[:,['brand_name','brand_slug']].drop_duplicates(['brand_name','brand_slug']).reset_index(drop=True)
brand = add_brand(brand)
brand['name_count'] = brand.apply(cal_brand_count,axis=1)
brand = brand.sort_values(by=['name_count'],ascending=False).reset_index(drop=True)
model = car_autohome_all.loc[:,['brand_name','brand_slug','model_name','model_slug']].drop_duplicates(['brand_name','brand_slug','model_name','model_slug']).reset_index(drop=True)
origin_train[['brand_name','brand_slug']] = origin_train.apply(find_brand,axis=1)
origin_train = origin_train.loc[(origin_train['brand_slug'].notnull()),:].reset_index(drop=True)
origin_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4995 entries, 0 to 4994
Data columns (total 14 columns):
source_id         4995 non-null int64
detail_name       4995 non-null object
year              4995 non-null float64
month             4995 non-null float64
mile              4995 non-null float64
price             4995 non-null float64
city              4995 non-null object
domain            4995 non-null object
transfer_owner    4995 non-null float64
online_year       4995 non-null object
volume            4210 non-null object
control           2713 non-null object
brand_name        4995 non-null object
brand_slug        4995 non-null object
dtypes: float64(5), int64(1), object(8)
memory usage: 546.4+ KB


In [28]:
f = open('./word_index.txt', 'r', encoding='UTF-8')
temp = f.read()
word_index = eval(temp)

with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

test_final = []
for i in range(0, len(origin_train)):
    temp = []
    x = origin_train['detail_name'][i]
    temp.extend(x)
    test_final.append(temp)
test_sequences = tokenizer.texts_to_sequences(test_final)

fill_zero = [0 for i in range(0,len(word_index))]

for i in range(0,len(origin_train)):  
    text = fill_zero.copy()
    for j in test_sequences[i]:
        text[j-1] = text[j-1] + 1
    origin_train.loc[i,'fill_vector'] = str(text)
origin_train = origin_train.rename(columns={'detail_name':'origin_name'})

In [29]:
def cal_cos_similar(df,y_vector):
    x_vector = eval(df['fill_vector'])
    y_vector = eval(y_vector)
    return np.dot(x_vector,y_vector) / (np.linalg.norm(x_vector,ord=2)*np.linalg.norm(y_vector,ord=2) )

def match_detail(df):
    print(df['brand_slug'],df['online_year'],df['control'],df['volume'])
    if (type(df['control']) != str) & math.isnan(float(df['volume'])):
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year'])),:].reset_index(drop=True)
    elif (type(df['control']) != str) & (not math.isnan(float(df['volume']))):
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year']))&(car_autohome_all['volume'] == float(df['volume'])),:].reset_index(drop=True)
    elif (type(df['control']) == str) & (math.isnan(float(df['volume']))):
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year']))&(car_autohome_all['control'] == df['control']),:].reset_index(drop=True)
    else:
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year']))&(car_autohome_all['control'] == df['control'])&(car_autohome_all['volume'] == float(df['volume'])),:].reset_index(drop=True)
    
    if len(temp_detail) == 0:
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year'])),:].reset_index(drop=True)
        if len(temp_detail) == 0:
            return pd.Series([np.NaN,np.NaN,np.NaN,np.NaN,np.NaN])
    temp_detail['cos_similar'] = temp_detail.apply(cal_cos_similar,args=(df['fill_vector'],),axis=1)
    temp_detail = temp_detail.loc[temp_detail.groupby(['brand_slug']).cos_similar.idxmax(), :]
    result = temp_detail.loc[temp_detail.groupby(['brand_slug']).cos_similar.idxmax(), ['model_slug','model_name','detail_slug','detail_name','cos_similar']].values[0]
    return pd.Series(result)

test = origin_train.copy()
test[['model_slug','model_name','detail_slug','detail_name','cos_similar']] = test.apply(match_detail,axis=1) 
test = test.drop(['year','month','mile','city','transfer_owner'],axis=1)

173 2015 自动 1.5
173 2015 自动 1.5
58 2014 nan 2.5
33 2010 自动 2.0
165 2016 自动 1.6
73 2016 nan 2.0
38 2012 nan 2.4
49 2017 nan 2.0
8 2013 nan 1.6
25 2016 自动 1.8
36 2012 nan nan
1 2013 nan 1.8
1 2013 自动 1.6
33 2014 nan nan
62 2016 自动 1.6
181 2014 手动 1.5
3 2012 手动 2.0
25 2017 nan 1.8
38 2016 nan nan
47 2016 nan nan
1 2017 自动 nan
3 2009 自动 2.0
14 2015 nan 2.4
1 2010 自动 1.8
8 2015 nan 1.5
75 2013 自动 1.5
70 2018 nan nan
46 2014 nan 2.4
38 2012 自动 1.8
46 2012 nan 2.0
36 2015 nan nan
71 2017 nan 2.4
46 2016 自动 1.4
71 2014 自动 1.6
15 2011 nan nan
181 2015 自动 2.0
63 2011 nan 2.0
73 2015 nan 2.0
46 2010 nan 2.4
38 2011 nan 2.4
71 2015 nan 2.4
13 2013 自动 2.0
13 2010 手动 1.6
20 2017 自动 1.5
1 2010 自动 1.8
33 2012 自动 2.0
67 2017 自动 nan
38 2013 手动 1.4
14 2011 nan 2.4
38 2012 nan 3.0
58 2014 自动 2.0
8 2015 nan 1.5
14 2012 nan 2.0
38 2010 nan 2.0
53 2016 手动 1.4
75 2017 nan 1.5
111 2015 手动 1.8
1 2010 自动 1.8
25 2016 nan 1.8
38 2012 自动 1.6
38 2016 自动 nan
49 2015 nan 2.0
8 2018 自动 2.0
8 2013 自动 1.5
67 2017 手动 1.6


1 2010 自动 1.8
56 2011 nan 1.6
12 2013 自动 2.4
8 2016 nan nan
25 2016 手动 1.5
14 2014 nan 2.4
56 2011 nan 1.6
1 2014 自动 1.8
56 2011 nan 1.6
25 2017 手动 1.5
19 2016 自动 nan
56 2011 nan 1.6
25 2016 自动 1.5
56 2011 nan 1.6
8 2016 nan nan
25 2017 手动 1.5
1 2012 自动 1.4
25 2017 手动 1.5
56 2011 nan 1.6
49 2012 nan 2.0
25 2013 手动 1.5
36 2009 nan nan
56 2011 nan 1.6
25 2013 手动 1.5
25 2013 手动 1.5
49 2012 nan 2.0
56 2011 nan 1.6
25 2018 自动 1.5
75 2014 nan 1.5
56 2011 nan 1.6
56 2011 nan 1.6
1 2012 自动 1.8
56 2011 nan 1.6
71 2013 自动 2.0
25 2013 手动 1.5
56 2011 nan 1.6
25 2013 手动 1.5
1 2011 自动 1.8
25 2013 手动 1.5
181 2017 自动 2.0
56 2011 nan 1.6
25 2013 手动 1.5
181 2017 自动 2.0
25 2013 手动 1.5
82 2017 nan nan
56 2011 nan 1.6
25 2012 nan 1.5
25 2012 nan 1.5
56 2011 nan 1.6
130 2014 nan 1.8
25 2012 nan 1.5
56 2012 nan 1.6
14 2010 自动 2.4
25 2012 nan 1.5
25 2012 nan 1.5
67 2014 自动 1.8
56 2011 nan 1.6
56 2012 nan 1.6
25 2012 nan 1.5
1 2009 手动 2.0
56 2012 nan 1.6
56 2012 自动 1.6
3 2011 自动 2.0
25 2008 手动 1.5
56 2012 nan 

25 2016 自动 1.8
45 2011 nan 1.0
122 2015 手动 1.6
13 2013 自动 2.0
122 2015 自动 1.6
45 2011 nan 1.0
1 2012 自动 1.8
38 2014 nan 2.4
45 2011 nan 1.0
122 2015 自动 1.6
122 2015 自动 1.6
47 2016 nan nan
15 2014 nan nan
45 2011 nan 1.0
122 2015 自动 1.6
45 2011 nan 1.0
122 2015 自动 1.6
38 2014 nan 2.4
45 2011 nan 1.0
122 2016 手动 1.6
181 2017 自动 2.0
38 2012 nan 2.4
45 2011 nan 1.0
122 2018 自动 1.6
38 2012 nan 2.0
45 2011 nan 1.0
122 2016 手动 1.6
15 2015 nan nan
45 2011 nan 1.0
8 2013 nan 1.6
38 2014 nan 2.0
45 2011 nan 1.0
122 2016 自动 1.6
45 2011 nan 1.0
122 2018 自动 1.6
73 2013 nan 2.5
45 2011 nan 1.0
1 2012 nan 1.8
45 2011 nan 1.0
122 2018 自动 1.6
45 2011 nan 1.0
27 2016 自动 2.3
45 2011 nan 1.0
122 2018 自动 1.6
65 2016 nan 2.5
45 2011 nan 1.0
45 2011 nan 1.0
1 2014 自动 1.8
45 2011 nan 1.0
122 2018 自动 1.6
8 2013 nan 2.0
45 2011 nan 1.0
122 2018 自动 1.6
122 2018 自动 1.6
122 2018 自动 1.6
15 2013 nan nan
45 2011 nan 1.0
122 2018 自动 1.6
46 2016 nan 2.4
122 2018 自动 1.6
15 2012 nan nan
1 2011 自动 1.8
45 2011 nan 1.0
122 

72 2015 手动 1.6
45 2012 nan 1.0
13 2015 自动 1.6
122 2013 自动 1.6
122 2014 手动 1.6
62 2012 自动 2.0
8 2016 nan nan
45 2012 nan 1.0
122 2014 手动 1.6
45 2012 nan 1.0
15 2015 nan nan
122 2014 手动 1.6
181 2018 自动 1.5
122 2013 手动 1.6
45 2012 nan 1.0
14 2012 nan 2.4
45 2013 nan 1.0
122 2014 手动 1.6
33 2015 自动 nan
122 2014 自动 1.6
169 2014 nan 1.6
45 2013 nan 1.0
12 2011 自动 2.0
36 2016 nan nan
45 2013 nan 1.0
1 2011 nan 2.0
122 2014 手动 1.6
45 2013 nan 1.0
33 2012 nan 1.4
122 2014 自动 1.6
36 2013 nan nan
45 2013 nan 1.0
12 2011 自动 2.0
45 2013 nan 1.0
122 2014 自动 1.6
14 2010 自动 2.4
45 2013 nan 1.0
122 2014 手动 1.6
45 2013 nan 1.0
122 2014 自动 1.6
122 2015 手动 1.6
45 2013 nan 1.0
45 2013 nan 1.0
122 2015 手动 1.6
45 2013 nan 1.0
122 2015 手动 1.6
45 2013 nan 1.0
122 2015 手动 1.6
45 2013 nan 1.0
122 2015 手动 1.6
122 2015 手动 1.6
45 2013 nan 1.0
122 2015 手动 1.6
122 2015 手动 1.6
45 2013 nan 1.0
45 2013 nan 1.0
122 2015 手动 1.6
45 2013 nan 1.0
45 2013 nan 1.0
45 2013 nan 1.0
45 2013 nan 1.0
45 2013 nan 1.0
45 2013 nan 1.0


122 2015 自动 2.0
283 2017 nan 2.0
1 2012 nan 1.8
283 2017 nan 2.0
122 2015 自动 2.0
49 2016 nan 2.0
122 2015 自动 2.0
283 2017 nan 2.0
1 2012 nan 1.8
122 2015 自动 2.0
13 2013 自动 1.6
14 2011 nan 2.0
122 2015 自动 2.0
283 2017 nan 2.0
122 2015 自动 2.0
8 2013 nan 2.0
283 2017 nan 2.0
283 2017 nan 2.0
122 2015 自动 2.0
26 2016 手动 1.5
283 2017 nan 2.0
84 2017 手动 2.0
283 2017 nan 2.0
8 2011 nan 2.0
122 2015 自动 2.0
283 2017 nan 2.0
38 2013 自动 1.5
122 2015 自动 2.0
283 2017 nan 2.0
71 2015 自动 1.3
122 2015 自动 2.0
122 2015 自动 2.0
283 2017 nan 2.0
13 2011 手动 1.6
122 2015 自动 2.0
71 2012 nan 1.6
122 2015 自动 2.0
26 2016 手动 1.5
283 2017 nan 2.0
1 2015 手动 1.4
283 2017 nan 2.0
122 2015 自动 2.0
46 2017 nan 2.4
283 2017 nan 2.0
33 2016 自动 nan
122 2015 自动 2.0
38 2012 nan 2.4
181 2014 手动 1.5
283 2017 nan 2.0
122 2015 自动 2.0
8 2016 nan nan
122 2015 自动 2.0
38 2012 nan 2.4
283 2017 nan 2.0
122 2015 自动 2.0
12 2013 自动 2.4
33 2012 nan nan
283 2017 nan 2.0
73 2016 nan 2.0
122 2015 自动 2.0
283 2017 nan 2.0
181 2017 自动 1.5
283 20

76 2018 自动 1.6
122 2017 自动 2.0
47 2013 nan 3.0
283 2017 nan 2.0
13 2011 自动 2.3
283 2017 nan 2.0
122 2017 自动 2.0
70 2010 nan 2.5
283 2017 nan 2.0
122 2017 自动 2.0
38 2015 nan 2.0
283 2017 nan 2.0
8 2013 手动 1.8
283 2017 nan 2.0
122 2017 自动 2.0
62 2017 自动 2.0
283 2017 nan 2.0
36 2017 nan nan
283 2017 nan 2.0
58 2013 自动 2.0
122 2017 自动 2.0
13 2014 手动 1.6
283 2017 nan 2.0
122 2017 自动 2.0
14 2012 nan 2.4
122 2017 自动 2.0
283 2017 nan 2.0
122 2017 自动 2.0
283 2017 nan 2.0
122 2017 自动 2.0
283 2017 nan 2.0
122 2017 自动 2.0
283 2017 nan 2.0
283 2017 nan 2.0
122 2017 自动 2.0
283 2017 nan 2.0
283 2017 nan 2.0
283 2017 nan 2.0
283 2017 nan 2.0
122 2017 自动 2.0
283 2017 nan 2.0
122 2017 自动 2.0
122 2017 自动 2.0
283 2017 nan 2.0
122 2017 自动 2.0
122 2017 自动 2.0
283 2017 nan 2.0
122 2017 自动 2.0
33 2014 nan nan
122 2017 自动 2.0
44 2017 nan 2.0
122 2017 自动 2.0
283 2017 nan 2.0
36 2012 nan nan
283 2017 nan 2.0
283 2017 nan 2.0
122 2017 自动 2.0
3 2009 nan 2.7
8 2011 nan 2.0
122 2017 自动 2.0
283 2017 nan 2.0
19 2010 手

71 2014 自动 2.0
110 2010 手动 1.3
181 2015 手动 1.5
181 2015 手动 1.5
15 2014 nan nan
181 2015 手动 1.5
76 2017 nan 2.0
110 2010 手动 1.3
181 2015 手动 1.5
65 2015 nan 2.0
82 2012 自动 2.0
1 2013 nan 1.8
181 2015 手动 1.5
38 2013 手动 1.5
110 2010 手动 1.3
113 2014 自动 1.5
181 2015 手动 1.5
8 2011 nan 2.3
110 2010 手动 1.3
181 2015 手动 1.5
110 2010 手动 1.3
13 2014 自动 1.6
181 2015 手动 1.5
181 2015 手动 1.5
12 2012 手动 2.0
110 2010 手动 1.3
110 2010 手动 1.3
181 2015 手动 1.5
181 2015 手动 1.5
14 2012 nan 2.0
110 2010 手动 1.3
15 2011 nan nan
110 2010 手动 1.3
110 2010 手动 1.3
181 2015 手动 1.5
46 2016 自动 1.4
110 2010 手动 1.3
181 2015 手动 1.5
181 2015 手动 1.5
1 2010 自动 1.8
49 2015 nan 2.0
181 2015 手动 1.5
1 2015 自动 1.6
3 2013 自动 2.0
181 2015 手动 1.5
58 2014 自动 1.5
25 2018 自动 1.8
181 2015 手动 1.5
47 2016 nan nan
130 2014 nan 1.8
181 2015 手动 1.5
86 2016 自动 1.5
110 2012 手动 1.3
25 2015 nan 2.4
181 2015 手动 1.5
110 2012 手动 1.5
181 2015 手动 1.5
36 2014 nan nan
38 2017 nan nan
181 2015 手动 1.5
110 2012 手动 1.5
181 2015 手动 1.5
14 2015 nan 2.4
110 2012

14 2016 自动 1.8
33 2016 nan nan
110 2017 手动 1.6
181 2016 手动 1.5
33 2016 nan nan
38 2014 nan 2.4
33 2016 nan nan
181 2016 手动 1.5
33 2016 nan nan
33 2016 nan nan
33 2016 nan nan
181 2016 手动 1.5
33 2016 nan nan
110 2017 自动 1.6
33 2016 nan nan
33 2016 nan nan
181 2016 手动 1.5
38 2018 nan nan
33 2016 nan nan
33 2016 nan nan
13 2014 自动 1.6
33 2016 nan nan
181 2016 手动 1.5
33 2015 自动 nan
33 2016 nan nan
33 2016 nan nan
110 2017 自动 1.6
33 2016 nan nan
181 2016 手动 1.5
33 2016 nan nan
62 2014 手动 2.0
33 2016 nan nan
110 2017 自动 1.6
33 2016 nan nan
181 2016 手动 1.5
33 2016 nan nan
8 2017 nan nan
33 2016 nan nan
181 2016 手动 1.5
110 2017 自动 1.6
33 2017 nan nan
33 2017 nan nan
181 2016 自动 1.5
110 2017 自动 1.6
33 2017 nan nan
33 2017 nan nan
33 2013 nan nan
181 2016 手动 1.5
33 2017 nan nan
76 2014 手动 2.0
33 2017 nan nan
33 2017 nan nan
181 2016 手动 1.5
110 2017 自动 1.6
33 2017 nan nan
33 2017 nan nan
181 2016 手动 1.5
1 2016 自动 nan
33 2017 nan nan
181 2016 手动 1.5
33 2017 nan nan
33 2017 nan nan
63 2010 手动 1.6
1

26 2012 手动 1.6
110 2016 手动 1.6
63 2010 手动 1.6
181 2014 手动 1.5
110 2016 自动 1.6
33 2017 nan nan
3 2009 nan 2.7
181 2014 手动 1.5
110 2016 自动 1.6
63 2010 手动 1.6
33 2017 nan nan
76 2017 nan 2.0
110 2016 自动 1.6
63 2010 手动 1.6
63 2010 手动 1.6
33 2017 nan nan
181 2014 手动 1.5
33 2017 nan nan
181 2014 手动 1.5
63 2010 手动 1.6
14 2015 nan 2.4
33 2017 nan nan
110 2016 手动 1.6
63 2010 手动 1.6
110 2017 nan nan
181 2014 手动 1.5
63 2010 手动 1.6
33 2018 nan nan
63 2010 手动 1.6
8 2013 nan 1.6
63 2010 自动 1.6
67 2012 手动 1.8
33 2018 nan nan
110 2016 自动 1.6
63 2010 自动 1.6
181 2014 手动 1.5
1 2011 自动 1.8
8 2013 nan 1.6
181 2014 手动 1.5
33 2018 nan nan
110 2018 手动 1.5
63 2010 自动 1.6
33 2018 nan nan
63 2010 自动 1.6
33 2018 nan nan
1 2011 自动 1.8
110 2015 手动 1.5
63 2010 自动 1.6
20 2012 手动 1.5
33 2018 nan nan
38 2012 nan 2.0
181 2014 手动 1.5
63 2010 自动 1.6
12 2015 nan 1.6
110 2018 手动 1.5
181 2014 手动 1.5
33 2018 nan nan
63 2010 自动 1.6
63 2010 自动 1.6
33 2018 nan nan
181 2014 手动 1.5
62 2016 自动 2.0
110 2015 手动 1.5
63 2010 自动 1.6
1 2

63 2013 手动 1.6
1 2012 自动 2.0
63 2013 手动 1.6
33 2018 nan nan
181 2014 手动 1.5
72 2011 手动 2.0
181 2014 手动 1.5
33 2018 nan nan
63 2013 手动 1.6
1 2012 自动 1.4
181 2014 手动 1.5
33 2018 nan nan
63 2013 手动 1.6
181 2014 手动 1.5
1 2012 nan 2.0
33 2018 nan nan
181 2014 手动 1.5
26 2017 自动 1.5
181 2014 手动 1.5
68 2013 手动 1.6
14 2017 自动 1.8
33 2018 nan nan
63 2013 手动 1.6
8 2015 nan 2.0
68 2013 手动 1.6
63 2013 手动 1.6
33 2018 nan nan
68 2013 手动 1.6
95 2016 自动 2.0
68 2013 手动 1.6
63 2013 手动 1.6
3 2016 自动 1.8
33 2018 nan nan
181 2014 手动 1.5
1 2015 手动 1.8
33 2018 nan nan
52 2015 nan nan
63 2013 手动 1.6
33 2018 nan nan
68 2013 手动 1.6
63 2013 手动 1.6
181 2014 手动 1.5
33 2018 nan nan
38 2010 nan 1.6
68 2013 手动 1.6
75 2011 手动 2.0
63 2013 自动 1.6
63 2013 手动 1.6
68 2013 手动 1.6
76 2017 自动 1.5
33 2018 nan nan
63 2013 手动 1.6
181 2014 手动 1.5
25 2016 nan 2.4
63 2013 手动 1.6
33 2018 nan nan
12 2013 自动 2.4
68 2013 手动 1.6
12 2015 自动 2.4
181 2014 手动 1.5
63 2013 自动 1.6
181 2014 手动 1.5
63 2013 自动 1.6
75 2013 手动 1.5
63 2013 手动 1.6
33 

In [24]:
test.to_csv('./man.csv',index=False)

In [30]:
print(len(test.loc[(test['cos_similar']>0.82),:]),len(test))

4986 4995


In [356]:
def process_price_bn(df):
    if df['price_bn'] == '暂无报价':
        return np.NAN
    temp = re.findall(r'万',df['price_bn'])
    if len(temp) > 1:
        text = df['price_bn'].split('~')[1]
    else:
        text = df['price_bn']
    text = text.replace('万','')
    return float(text)

def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """    
    text = df[column_name]
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('-', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

# 判断是否有重复final_text
origin_train = pd.read_csv('./origin_train.csv')
origin_train = origin_train.loc[(origin_train['domain'] == 'guazi.com'),:].reset_index(drop=True).head(10000)

# origin_train = origin_train.loc[(origin_train['source_id'].notnull())&(origin_train['detail_name'].notnull())&(origin_train['price'] > 0)&(origin_train['transfer_owner'] == 0), :].reset_index(drop=True)
origin_train['title'] = origin_train.apply(delete_str_useless, args=('detail_name',), axis=1)
origin_train = origin_train.rename(columns={'detail_name':'origin_name'})
train_final = pd.read_csv('../tmp/train/train_final.csv')
train_final = train_final.rename(columns={'final_text':'title'})

# data = wait_predict.loc[(wait_predict['domain'].isin(['guazi.com'])), :]
data = origin_train.copy()
print('瓜子车源数据量:', len(data))
final = data.merge(train_final, how='left', on=['title'])
final = final.sort_values(by=['title'])
match = final.loc[(final['online_year'].notnull())&(final['city'].notnull()), :].reset_index(drop=True)
print('瓜子车源数据匹配量:',len(match), '匹配率:', len(match)/len(data))
match.info()

瓜子车源数据量: 10000
瓜子车源数据匹配量: 9465 匹配率: 0.9465
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9465 entries, 0 to 9464
Data columns (total 18 columns):
source_id         9465 non-null int64
origin_name       9465 non-null object
year              9465 non-null int64
month             9465 non-null float64
mile              9465 non-null float64
price             9465 non-null float64
city              9465 non-null object
domain            9465 non-null object
transfer_owner    9465 non-null float64
title             9465 non-null object
brand_slug        9465 non-null float64
model_slug        9465 non-null float64
detail_slug       9465 non-null float64
online_year       9465 non-null float64
price_bn          9465 non-null float64
brand_name        9465 non-null object
model_name        9465 non-null object
detail_name       9465 non-null object
dtypes: float64(9), int64(2), object(7)
memory usage: 1.3+ MB


In [362]:
man = test.copy()
man = man.merge(match.loc[:,['source_id','detail_slug']].rename(columns={'detail_slug':'corr_slug'}),how='left',on=['source_id'])
man = man.loc[(man['corr_slug'].notnull()),:].reset_index(drop=True)
equal = man.loc[(man['corr_slug'] == man['detail_slug']),:].reset_index(drop=True)
print(len(equal)/len(man))
man.info()

0.9990491283676703
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9465 entries, 0 to 9464
Data columns (total 16 columns):
source_id      9465 non-null int64
origin_name    9465 non-null object
price          9465 non-null float64
domain         9465 non-null object
online_year    9465 non-null object
volume         7824 non-null object
control        5314 non-null object
brand_name     9465 non-null object
brand_slug     9465 non-null object
fill_vector    9465 non-null object
model_slug     9465 non-null float64
model_name     9465 non-null object
detail_slug    9465 non-null float64
detail_name    9465 non-null object
cos_similar    9465 non-null float64
corr_slug      9465 non-null float64
dtypes: float64(5), int64(1), object(10)
memory usage: 1.2+ MB


In [363]:
man.loc[(man['corr_slug'] != man['detail_slug']),:].reset_index(drop=True)

,source_id,origin_name,price,domain,online_year,volume,control,brand_name,brand_slug,fill_vector,model_slug,model_name,detail_slug,detail_name,cos_similar,corr_slug
0,3007509740,日产蓝鸟2016款1.6lcvt炫酷版,8.32,guazi.com,2016,1.6,自动,日产,63,"[1, 1, 2, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0, ...",425.0,骐达,26344.0,2016款 1.6L CVT酷动版,0.869565,24002.0
1,3006883150,起亚k52012款2.0l自动dlx,8.56,guazi.com,2012,2.0,自动,起亚,62,"[2, 3, 1, 1, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...",2246.0,起亚k5,12220.0,2012款 2.0L 自动DLX-2,0.973964,12219.0
2,3007347629,起亚k52012款2.0l自动dlx,7.89,guazi.com,2012,2.0,自动,起亚,62,"[2, 3, 1, 1, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...",2246.0,起亚k5,12220.0,2012款 2.0L 自动DLX-2,0.973964,12219.0
3,3007648651,起亚k52012款2.0l自动dlx,8.40,guazi.com,2012,2.0,自动,起亚,62,"[2, 3, 1, 1, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...",2246.0,起亚k5,12220.0,2012款 2.0L 自动DLX-2,0.973964,12219.0
4,3007759413,起亚k52012款2.0l自动dlx,8.93,guazi.com,2012,2.0,自动,起亚,62,"[2, 3, 1, 1, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...",2246.0,起亚k5,12220.0,2012款 2.0L 自动DLX-2,0.973964,12219.0
5,3007776394,日产蓝鸟2016款1.6lcvt炫酷版,8.79,guazi.com,2016,1.6,自动,日产,63,"[1, 1, 2, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0, ...",425.0,骐达,26344.0,2016款 1.6L CVT酷动版,0.869565,24002.0
6,3007818797,起亚k52012款2.0l自动dlx,7.91,guazi.com,2012,2.0,自动,起亚,62,"[2, 3, 1, 1, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...",2246.0,起亚k5,12220.0,2012款 2.0L 自动DLX-2,0.973964,12219.0
7,3007976681,日产蓝鸟2016款1.6lcvt炫酷版,8.95,guazi.com,2016,1.6,自动,日产,63,"[1, 1, 2, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0, ...",425.0,骐达,26344.0,2016款 1.6L CVT酷动版,0.869565,24002.0
8,3007831895,起亚k52012款2.0l自动dlx,7.71,guazi.com,2012,2.0,自动,起亚,62,"[2, 3, 1, 1, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...",2246.0,起亚k5,12220.0,2012款 2.0L 自动DLX-2,0.973964,12219.0


In [371]:
x_vector = eval(man.loc[(man['source_id'] == 3006883150),'fill_vector'].values[0])
y_vector = eval(car_autohome_all.loc[(car_autohome_all['detail_slug'] == 12220),'fill_vector'].values[0])
np.dot(x_vector,y_vector) / (np.linalg.norm(x_vector,ord=2)*np.linalg.norm(y_vector,ord=2))

0.9739642045665762

In [372]:
car_autohome_all.loc[(car_autohome_all['detail_slug'] == 12220),:]

,detail_slug,brand_name,brand_slug,model_name,model_slug,detail_name,manufacturer,price_bn,body,energy,volume_extend,emission_standard,listed_year,online_year,volume,control,final_text,fill_vector
14488,12220,起亚,62,起亚k5,2246,2012款 2.0L 自动DLX-2,东风悦达起亚,19.98,中型车,汽油,L,国IV(国V),2012,2012,2.0,自动,起亚起亚k52012款2.0l自动dlx2,"[2, 4, 1, 1, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."


In [373]:
car_autohome_all.loc[(car_autohome_all['detail_slug'] == 12219),:]

,detail_slug,brand_name,brand_slug,model_name,model_slug,detail_name,manufacturer,price_bn,body,energy,volume_extend,emission_standard,listed_year,online_year,volume,control,final_text,fill_vector
14487,12219,起亚,62,起亚k5,2246,2012款 2.0L 自动DLX,东风悦达起亚,18.88,中型车,汽油,L,国IV(国V),2012,2012,2.0,自动,起亚起亚k52012款2.0l自动dlx,"[2, 3, 1, 1, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."


In [ ]:
#----------------
# 1.车系余弦相似匹配
# 2.车系+款型余弦相似匹配

In [26]:
f = open('../tmp/train/word_index.txt', 'r', encoding='UTF-8')
temp = f.read()
word_index = eval(temp)
word_index = pd.DataFrame.from_dict(word_index, orient='index').reset_index()
word_index.columns = ['word','num']
word_index

,word,num
0,穿,555
1,雪,111
2,乐,203
3,隐,860
4,丰,122
5,骊,487
6,簧,655
7,名,193
8,渡,563
9,罗,272


In [18]:
tokenizer

In [20]:
car_autohome_cos_vector = pd.read_csv('../tmp/train/car_autohome_cos_vector.csv')
car_autohome_cos_vector.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32672 entries, 0 to 32671
Data columns (total 21 columns):
detail_slug          32672 non-null int64
brand_name           32672 non-null object
brand_slug           32672 non-null int64
model_name           32672 non-null object
model_slug           32672 non-null int64
detail_name          32672 non-null object
manufacturer         32672 non-null object
price_bn             32672 non-null float64
body                 32672 non-null object
energy               32672 non-null object
emission_standard    32672 non-null object
listed_year          32672 non-null int64
online_year          32672 non-null int64
volume               32672 non-null float64
control              32672 non-null object
fill_vector          32672 non-null object
gpj_model_slug       32672 non-null object
gpj_detail_slug      32672 non-null object
gpj_brand_slug       32672 non-null object
body_style           29779 non-null object
popular              32672 non-nul

In [15]:
import json
test = {'message': '', 'data': {'control': '自动', 'energy': '汽油', 'online_year': 2016, 'price_bn': 10.19, 'gpj_model_slug': 'jiliyuanjingSUV', 'cos_similar': 0.8563488385776753, 'body_style': 'compact_suv', 'detail_name': '2016款 1.3T CVT旗舰型', 'gpj_brand_slug': 'car_autohome_brand_25', 'gpj_detail_slug': '111239_autotis', 'emission_standard': '国V', 'gpj_index': 5.42, 'volume': 1.3, 'eval_price': 6.9189, 'model_name': '远景SUV', 'brand_name': '吉利汽车', 'rank': 541996244.0, 'body': '紧凑型SUV', 'listed_year': 2016, 'popular': 'B', 'manufacturer': '吉利汽车'}, 'status': 'success'}
test = json.dumps(test)
test

'{"status": "success", "message": "", "data": {"volume": 1.3, "brand_name": "\\u5409\\u5229\\u6c7d\\u8f66", "gpj_brand_slug": "car_autohome_brand_25", "manufacturer": "\\u5409\\u5229\\u6c7d\\u8f66", "listed_year": 2016, "energy": "\\u6c7d\\u6cb9", "body_style": "compact_suv", "rank": 541996244.0, "price_bn": 10.19, "detail_name": "2016\\u6b3e 1.3T CVT\\u65d7\\u8230\\u578b", "gpj_model_slug": "jiliyuanjingSUV", "model_name": "\\u8fdc\\u666fSUV", "emission_standard": "\\u56fdV", "body": "\\u7d27\\u51d1\\u578bSUV", "gpj_index": 5.42, "cos_similar": 0.8563488385776753, "control": "\\u81ea\\u52a8", "popular": "B", "eval_price": 6.9189, "gpj_detail_slug": "111239_autotis", "online_year": 2016}}'